# このドキュメントの目的

半手動で行うテキストへのタグ付けプロセスを、その手順を含めてまとめること。

# 必要な環境情報

- MacOS
- python3.6
- anaconda
- mecab-python3

# 手順の概要

### オペレーション_その１

1. はじめにドキュメントをファイルに分けて用意します。
2. 読み込んだデータをMeCabで形態素解析して品詞分解する。(この品詞分解には独自にカスタマイズしたモジュールを使っています)
3. 2で検出した単語（＝品詞の自然活用系（ ×:飲める → ○：飲む ））を、スペース区切り文字列（わかち書き）に変換。
4. 3の結果をsklearnのtopic_modelを実行します。
5. 各ドキュメントの予測トピックカテゴリをファイルとして出力します。

### オペレーション_その2

1. オペレーション_その１で出力されたテキストのカテゴライズデータをエクセルで見ながら、手動でtagをつけたファイルを作成します。（半手動部分）
2. この時、全てのテキストにtagをつける必要はありません。パッと見て判断がついたものだけにします。
3. 手動でtagづけされたデータを再学習します。
4. まだtagを手動でつけていないテキストも含めて、tagを予測します。
5. 予測タグのデータがファイル出力されますので、間違った予測タグを修正します。
6. 修正したファイルを、3に入力し直して、もう一度再学習します。
7. 以下、3~6を繰り返して、精度を改善します。


In [1]:
from module.mecab_wrapper import keitaiso
import os
import re


In [2]:
# ドキュメントファイルの読み込み
DIR_TEXT_FILE = 'data/text/topic-news'
doc_files = [os.path.join(DIR_TEXT_FILE, f) for f in os.listdir(DIR_TEXT_FILE) if re.match('^topic-news-.*.txt$', f)]
doc_texts = list()

# テキストの前処理関数を定義する
def remove_top2line(text):
    #　初めの2行は記事のURLと掲載開始日時なので省く
    text = ' '.join(text.split('\n')[2:])    
    return text
    
def text_cleaning(text):
    transform_dict = {',':'、', '\t':' '}
    for f,t in transform_dict.items():
        text.replace(f, t)
    return text

for df in doc_files:
    with open(df, 'r') as f:
        text = f.read()
        text = remove_top2line(text)
        text = text_cleaning(text)
        doc_texts.append(text)

# 中身の確認
doc_files[0]
print( doc_texts[0] )

悪評が次から次へと溢れ出る川越シェフ 最近、TVでも人気者の川越シェフ。代官山のおしゃれなイタリアンレストランのオーナーシェフでありながら、タレントとも軽妙なトークを交わすマルチな才能で、TV業界でもその地位を不動のものとしつつある。また、その甘いマスクに魅せられた、「川越女子」なる固定ファンまで存在するという。  しかし、ここにきて川越シェフの悪評がどんどん噴出しているようだ。最近では、ミニストップの川越シェフプロデュースクリスマスケーキが、1万円という超高額で売り出されることが話題となった。  【関連記事】 ミニストップの川越シェフプロデュースクリスマスケーキがバカ高いと話題に http://news.livedoor.com/topics/detail/5899457/  普通のケーキなら、原価は200円程度。川越シェフプロデュースだけに、多少豪華に作ってあるかもしれないが、それでも「ぼったくり」というネット上の反応が多かった。また、それ以上に、「有名になった途端、ここぞとばかりに金儲けしている」という声も散見された。  また、普段のマスコミ対応の悪さもかなりのものだ。数年前までは、川越シェフなどほとんど誰も知らないような状態だったが、有名となった今は手の平を返すように、週刊誌の記者なを見下すような態度が見られるようになったという。  マスコミのおかげで有名になっておきながら、恩知らずとはこのことだろう。  【関連記事】 シェフのタレント化はもうたくさん!?　マスコミが嫌うタレントシェフ http://news.livedoor.com/topics/detail/5801160/  しかし、単なるシェフ風情が調子に乗ってしまうのも無理はない。何せ、「株式会社TATSUYA KAWAGOE」の年商は約9,000万円、さらにTVへの出演料も高額に上る。  また、6月にTV番組で「バツ2」の独身であることを告白している。あの甘いマスクに有名人とくれば、モテないはずはない。つまり、カネもオンナも“やりたい放題”なのだろう。  【関連記事】 離婚告白川越達也シェフ　週1ネイルサロン、収録時は口紅も http://news.livedoor.com/topics/detail/5655259/  川越シェフの15才年下美人秘書「シェフとデキてる」説を否定 http://

In [3]:
# 自作のmecab wrapperで形態素解析の設定を行う。
# この形態素解析のwrapperでは、動詞は自動的に自然活用形に変換される。

kei = keitaiso(use_PoW=['名詞'], stop_words=['彼女', '彼', '私', 'する', '*'])

# 以下、実行サンプル
kei.tokenize(doc_texts[0])

[[0, '名詞', '悪評'],
 [2, '名詞', '次'],
 [4, '名詞', '次'],
 [9, '名詞', '川越'],
 [10, '名詞', 'シェフ'],
 [11, '名詞', '最近'],
 [16, '名詞', '人気'],
 [17, '名詞', '者'],
 [19, '名詞', '川越'],
 [20, '名詞', 'シェフ'],
 [22, '名詞', '代官山'],
 [24, '名詞', 'おしゃれ'],
 [26, '名詞', 'イタリアン'],
 [27, '名詞', 'レストラン'],
 [29, '名詞', 'オーナー'],
 [30, '名詞', 'シェフ'],
 [35, '名詞', 'タレント'],
 [37, '名詞', '軽妙'],
 [39, '名詞', 'トーク'],
 [42, '名詞', 'マルチ'],
 [44, '名詞', '才能'],
 [48, '名詞', '業界'],
 [52, '名詞', '地位'],
 [54, '名詞', '不動'],
 [56, '名詞', 'もの'],
 [66, '名詞', 'マスク'],
 [73, '名詞', '川越'],
 [74, '名詞', '女子'],
 [77, '名詞', '固定'],
 [78, '名詞', 'ファン'],
 [80, '名詞', '存在'],
 [87, '名詞', 'ここ'],
 [91, '名詞', '川越'],
 [92, '名詞', 'シェフ'],
 [94, '名詞', '悪評'],
 [97, '名詞', '噴出'],
 [101, '名詞', 'よう'],
 [104, '名詞', '最近'],
 [108, '名詞', 'ミニストップ'],
 [110, '名詞', '川越'],
 [115, '名詞', '万'],
 [116, '名詞', '円'],
 [119, '名詞', '高額'],
 [123, '名詞', 'こと'],
 [125, '名詞', '話題'],
 [131, '名詞', '関連'],
 [132, '名詞', '記事'],
 [134, '名詞', 'ミニストップ'],
 [136, '名詞', '川越'],
 [139, '名詞', 'バカ'],
 [142, '名詞', '話題

In [4]:
# doc_textsをスペース区切り文字列に変換する。
convert_wakachi = lambda text:  ' '.join([token[2] for token in kei.tokenize(text)])
doc_wakachis = [convert_wakachi(text) for text in doc_texts]

doc_wakachis[0]

'悪評 次 次 川越 シェフ 最近 人気 者 川越 シェフ 代官山 おしゃれ イタリアン レストラン オーナー シェフ タレント 軽妙 トーク マルチ 才能 業界 地位 不動 もの マスク 川越 女子 固定 ファン 存在 ここ 川越 シェフ 悪評 噴出 よう 最近 ミニストップ 川越 万 円 高額 こと 話題 関連 記事 ミニストップ 川越 バカ 話題 普通 ケーキ 原価 円 程度 川越 シェフ プロデュース 豪華 くり ネット 上 反応 それ 以上 有名 途端 ここ 金儲け 声 散見 普段 マスコミ 対応 さ かなり もの 数 年 前 川越 シェフ 誰 よう 状態 有名 今 手の平 よう 週刊 誌 記者 よう 態度 よう マスコミ おかげ 有名 恩知らず こと 関連 記事 シェフ タレント 化 たくさん マスコミ タレント シェフ シェフ 風情 調子 の 無理 株式会社 年商 万 円 出演 料 高額 月 番組 バツ 独身 こと 告白 マスク 有名人 はず カネ 放題 の 関連 記事 離婚 告白 川越 達也 シェフ 週 収録 時 口紅 川越 シェフ 才 年下 美人 秘書 シェフ 説 否定 天狗 っぷり 盛ん 報道 よう 最近 収録 現場 こと よう 現場 スタッフ 評判 の 関連 記事 川越 シェフ 我慢 限界 あと 肝心 料理 腕 地下 惣菜 レベル 意見 それ 上 目線 評論 疑問符 化けの皮 十分 本人 の 関連 記事 大人気 川越 達也 氏 カネ 評判 川越 シェフ 程度 買い 視聴 者 レベル 問題 メディア 戦略 よう 頭 残念 人 言葉 日本 景気 ため パー お金 もの 関連 記事 川越 達也 プロデュース 商品 件 川越 存在 川越 達也 毎日 川越 達也 監修 商品 川越 女子 川越 達也 メニュー 川越 シェフ キムチ 川越 達也 顔 川越 女子 気 気 川越 達也 シェフ'

In [5]:
# doc - word matrix　を作成する（bags of words, 単なる出現数カウント）
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer =  CountVectorizer(
    max_df=0.50, # 50%以上のdocに含まれる単語は無視する。
    min_df=0.05, # 5%以下のdocにしか含まれない単語は無視する。    
    )
doc_word_matrix = count_vectorizer.fit_transform(doc_wakachis)

# 学習済みの　count_vectorizer はpickleで保存しておく。
import pickle
file_name = 'data/processed/count_vectorizer.pickle'
with open(file_name, 'wb') as f:
    pickle.dump(count_vectorizer, f)


In [6]:
# topic model (LatentDirichletAllocation) のモジュールを読み込む
from sklearn.decomposition import LatentDirichletAllocation

N_TOPIC = 3
lda = LatentDirichletAllocation(n_components=N_TOPIC, random_state=0)

# 学習して保存しておく。
lda.fit(doc_word_matrix)
file_name = 'data/processed/lda.pickle'
with open(file_name, 'wb') as f:
    pickle.dump(lda, f)

# 分類された topic
import pandas as pd
df_doc_topic = pd.DataFrame(
    lda.transform(doc_word_matrix), 
    columns=['topic%02d'%i for i in range(N_TOPIC)]
)
df_doc_topic['doc_file'] = doc_files
df_doc_topic['doc_text'] = doc_texts

df_doc_topic

,topic00,topic01,topic02,doc_file,doc_text
0,0.783779,0.048281,0.167940,data/text/topic-news/topic-news-5903225.txt,悪評が次から次へと溢れ出る川越シェフ 最近、TVでも人気者の川越シェフ。代官山のおしゃれなイ...
1,0.991185,0.004218,0.004596,data/text/topic-news/topic-news-5903373.txt,写真が流出し“堕ちていった”アイドルたち アイドルが男性と一緒に写ったプリクラやニャンニャン...
2,0.697792,0.007239,0.294969,data/text/topic-news/topic-news-5907376.txt,紳助とたけしの明暗を分けたものとは 今年で20回目を迎えたTBSのオールスター感謝祭に、1回...
3,0.726012,0.075422,0.198567,data/text/topic-news/topic-news-5908396.txt,ガールズバンドSCANDAL、今年着た水着を告白 9月24日から10月2日までの4日間、お台...
4,0.841284,0.005133,0.153583,data/text/topic-news/topic-news-5910461.txt,読売テレビ広報部長の無礼千万メールにジャーナリスト烏賀陽（うがや）弘道さんが激怒 ジャー...
5,0.019118,0.303659,0.677223,data/text/topic-news/topic-news-5910462.txt,iPhone5、auに独占供給か 米国版ウォールストリートジャーナルが報じたiPhone5に...
6,0.021851,0.956097,0.022052,data/text/topic-news/topic-news-5911453.txt,“辛ラーメンにゴキブリ”ネットで話題に 日本でも目にすることが多くなった韓国のインスタントラ...
7,0.301523,0.626453,0.072024,data/text/topic-news/topic-news-5913367.txt,韓国がハブられたiPhone4S発表会 米アップルが4日に発表したiPhone4S。iPh...
8,0.125260,0.017644,0.857095,data/text/topic-news/topic-news-5913447.txt,GREEの退会引き止めにユーザーの不満が爆発 グリー株式会社が運営するSNS「GREE」の退...
9,0.979616,0.008712,0.011671,data/text/topic-news/topic-news-5914835.txt,毎年話題になるベストジーニスト賞の偏り 今年のベストジーニスト一般選出部門に相葉雅紀、黒木メ...


In [7]:
# df_doc_topic をファイル出力する。
file_name = 'data/df_doc_topic.csv'
df_doc_topic.to_csv(file_name, index=False)



# 以上で、オペレーション_01 は終了です


# 続いて、オペレーション_02 を行います


In [8]:
# 手続き_02で、手動でtagをつける作業を行います。
# tagをつけたデータを読み込みます。
# 'data/tag.txt' に手動でタグをつけたデータを読み込みます。
TAG_FILE = 'data/tag.txt'
try:
    df_tag = pd.read_csv(TAG_FILE, sep='\t')
except:
    df_tag = pd.read_csv(TAG_FILE, sep='\t', encoding="932")

# このように手動でtagをつけたものです。
# 全てのtextにtagをつける必要はありません。
df_tag


,topic00,topic01,topic02,doc_file,doc_text,tag_芸能,tag_海外,tag_社会,tag_政治,tag_スポーツ
0,0.006954,0.986436,0.006611,data/text/topic-news/topic-news-6843698.txt,同じホテルに滞在している、日本代表と韓国代表 10日、韓国「ソウル新聞」は、翌11日（日本時...,0.0,1.0,0.0,0.0,1.0
1,0.007343,0.985629,0.007028,data/text/topic-news/topic-news-6841012.txt,五輪サッカー日韓戦において、韓国が注目する“ある選手”とは？ 日本時間11日午前3時45分よ...,0.0,1.0,0.0,0.0,1.0
2,0.007180,0.985483,0.007337,data/text/topic-news/topic-news-6829539.txt,女子アーチェリー銅メダリスト早川蓮、 韓国でも話題に 6日、アーチェリー女子銅メダリストの早...,0.0,1.0,0.0,0.0,1.0
3,0.007506,0.984469,0.008026,data/text/topic-news/topic-news-6608684.txt,五輪逃したタイが「日本の八百長疑惑」を一斉報道 29日、タイでは、女子バレーボール五輪予選で...,0.0,1.0,0.0,0.0,1.0
4,0.007822,0.984203,0.007975,data/text/topic-news/topic-news-6842234.txt,五輪サッカーで日韓が激突、韓国ユーザは「餌を出して試合させるのは間違っている」 韓国「文化日...,0.0,1.0,1.0,0.0,1.0
5,0.007915,0.983509,0.008577,data/text/topic-news/topic-news-6818120.txt,女子バドミントン失格に韓国では「なでしこジャパンの引き分け狙いはOKなのになぜ…」の声 ロン...,0.0,1.0,0.0,0.0,1.0
6,0.008171,0.983338,0.008491,data/text/topic-news/topic-news-6636116.txt,FIFA最新ランキング発表、韓国ユーザが敗北宣言!? 6日、FIFAはサッカーの世界ランキン...,0.0,1.0,0.0,0.0,1.0
7,0.008762,0.982255,0.008983,data/text/topic-news/topic-news-6782959.txt,【韓国ニュース】WBC日本不参加の報道に「韓国の優勝か!?」 20日、労働組合・日本プロ野球...,0.0,1.0,0.0,0.0,1.0
8,0.008930,0.981640,0.009430,data/text/topic-news/topic-news-6829622.txt,【五輪サッカー】 韓国に負けたイギリス代表に英ネットユーザーから批判相次ぐ 5日、イギリスの...,0.0,1.0,0.0,0.0,1.0
9,0.009588,0.980599,0.009813,data/text/topic-news/topic-news-6822413.txt,五輪サッカー 英韓戦を前に韓国では「最悪のシナリオ」の声 2日、韓国のニュースサイト「イデ...,0.0,1.0,0.0,0.0,1.0


In [9]:
# 後半のほとんどが未記入です。
df_tag[100:]

,topic00,topic01,topic02,doc_file,doc_text,tag_芸能,tag_海外,tag_社会,tag_政治,tag_スポーツ
100,0.017635,0.821989,0.160377,data/text/topic-news/topic-news-6017374.txt,日本の習慣に外国人が激怒 居酒屋で注文せずとも出される“お通し”。日本ではお馴染みの習慣だが...,NaN,NaN,NaN,NaN,NaN
101,0.016988,0.821360,0.161651,data/text/topic-news/topic-news-5966020.txt,日本スケート連盟のキム・ヨナ締め出しに「当然」の声 「キム・ヨナが日本のアイスショーからは...,NaN,NaN,NaN,NaN,NaN
102,0.010429,0.818316,0.171255,data/text/topic-news/topic-news-6388021.txt,中国警察が女子高生を生きたまま田んぼに遺棄 21日、朝日新聞は、中国の警察が倒れている女子高...,NaN,NaN,NaN,NaN,NaN
103,0.014804,0.816421,0.168774,data/text/topic-news/topic-news-6226414.txt,「日本人が最も食べている漬物はキムチ」に「そんなわけないだろ」 25日、YOMIURI ON...,NaN,NaN,NaN,NaN,NaN
104,0.008732,0.815220,0.176047,data/text/topic-news/topic-news-6395897.txt,中国提案の「南京大虐殺否定罪」に、日本では「幼稚で横暴」だと非難殺到 21日、NEWSポスト...,NaN,NaN,NaN,NaN,NaN
105,0.010536,0.814796,0.174668,data/text/topic-news/topic-news-6705586.txt,【韓国ニュース】北朝鮮が日韓の軍事協定を批判 韓国ユーザーからは「敵の敵は味方」の声 28...,NaN,NaN,NaN,NaN,NaN
106,0.014255,0.808532,0.177213,data/text/topic-news/topic-news-6349177.txt,バットの上に長時間正座…プロ野球界の主従関係を非難 7日、プロ野球独立リーグのBCリーグ「福...,NaN,NaN,NaN,NaN,NaN
107,0.031740,0.807555,0.160705,data/text/topic-news/topic-news-6696339.txt,フィリピンで日本人女性2人に性的暴行を加えた韓国人を逮捕 韓国ネットユーザーからは「日本は昔...,NaN,NaN,NaN,NaN,NaN
108,0.008011,0.803337,0.188652,data/text/topic-news/topic-news-6353761.txt,「勝手な広告出すな!」韓国への感謝広告に批判殺到 \t 9日、在韓国日本大使館が各韓国紙に「...,NaN,NaN,NaN,NaN,NaN
109,0.004974,0.802660,0.192365,data/text/topic-news/topic-news-5934315.txt,ピザの起源は“韓国”とするCMが話題に ピザの起源を韓国だとする広告動画を韓国のピザ会社「ミ...,NaN,NaN,NaN,NaN,NaN


In [10]:
# ここで、textからベクトルを生成する関数を定義しておきます。

text = '''
衆議院の総選挙が開始しました。
事前の電話調査によれば、与党、野党ともに指示が割れていて予断ができない状態です。
'''

## テキストをtopicに変換するために必要な学習済みインスタンスを読み込む。
file_name = 'data/processed/count_vectorizer.pickle'
with open(file_name, 'rb') as f:
    count_vectorizer = pickle.load(f)
file_name = 'data/processed/lda.pickle'
with open(file_name, 'rb') as f:
    lda = pickle.load(f)

## text_vectorize関数を作成し、textをtopicベクトルに変換する手続きを定義する。
import numpy as np
def text_vectorize(text):
    _text = text_cleaning(text)
    token = kei.tokenize(_text)
    wakachi = ' '.join([to[2] for to in token])
    count_vec = count_vectorizer.transform([wakachi])[0]
    topic_vec = lda.transform(count_vec)
    # 今回は、topic_vecとcount_vec を変数とする　（tf-idfでもよい）
    return np.concatenate([topic_vec, count_vec.toarray()], axis=1)[0]

## このように、入力されたtextをtopicベクトルとして出力することができるようになりました。
x = text_vectorize(text)
x



array([0.18183837, 0.17303968, 0.64512195, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [11]:
# 手動でつけたタグを学習します。
from sklearn.ensemble import GradientBoostingClassifier

estimater_dict = dict()
tags    = [col for col in df_tag.columns if re.match('^tag_', col)] 
topics = [col for col in df_tag.columns if re.match('^topic', col)] 
for tag in tags:
    # tag を未記入の部分は学習から除外します
    train_index = np.logical_not(np.isnan(df_tag[tag]))
    y = df_tag.loc[train_index, tag]
    # 上記で定義したベクトル化関数を使います。
    X = [text_vectorize(text) for text in df_tag.loc[train_index, 'doc_text']]
    estimator = GradientBoostingClassifier()
    estimator.fit(X, y)
    estimater_dict[tag] = estimator

# タグごとの学習済みセットが出来上がります。
estimater_dict

{'tag_芸能': GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.1, loss='deviance', max_depth=3,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100,
               n_iter_no_change=None, presort='auto', random_state=None,
               subsample=1.0, tol=0.0001, validation_fraction=0.1,
               verbose=0, warm_start=False),
 'tag_海外': GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.1, loss='deviance', max_depth=3,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100,
               n_iter_no_change=None, presort='auto', r

In [12]:
# では新しいtextに対して予測させてみます。
text = '''
衆議院の総選挙が開始しました。
事前の電話調査によれば、与党、野党ともに指示が割れていて予断ができない状態です。
'''
def predict(text):
    x = text_vectorize(text)
    predict_result = {}
    for tag, estimator in estimater_dict.items():
        predict_result[tag] = estimator.predict_proba([x])[0][1]

    return predict_result

predict(text) # 政治は予測できているようです。

{'tag_芸能': 0.00017465496603007933,
 'tag_海外': 0.00029488326534318207,
 'tag_社会': 0.0006311296630511109,
 'tag_政治': 0.9992325150247189,
 'tag_スポーツ': 0.00017988462140266926}

In [13]:
text = '''
先日、サッカーの世界大会がありました。
日本代表チームは予選突破し、決勝トーナメントへ駒を進めました。しかし、惜しくも３回戦で敗退しました。
次は五輪オリンピック予選に向けて、調整に入ります。
'''
predict(text) # 海外は予測できているけれど、スポーツはまだ不十分です。

{'tag_芸能': 0.00017465496603007933,
 'tag_海外': 0.999705116734657,
 'tag_社会': 0.0006311296630511109,
 'tag_政治': 0.00017797109066622262,
 'tag_スポーツ': 0.00017988462140266926}

In [14]:
# より予測精度を上げるためには、1)モデルの改善、2)データの改善を行います。
# ここではデータの改善に取り組んでみます。　モデルはいつでも変更できますが、tagつきデータはどこにもありません。

# まずは、現状の予測 数値を　df_tag に追加します。
for index, row in df_tag.iterrows():
    predicted_dict = predict(row['doc_text'])
    for tag, val in predicted_dict.items():
        df_tag.loc[index, 'predict_'+tag] = val

df_tag

,topic00,topic01,topic02,doc_file,doc_text,tag_芸能,tag_海外,tag_社会,tag_政治,tag_スポーツ,predict_tag_芸能,predict_tag_海外,predict_tag_社会,predict_tag_政治,predict_tag_スポーツ
0,0.006954,0.986436,0.006611,data/text/topic-news/topic-news-6843698.txt,同じホテルに滞在している、日本代表と韓国代表 10日、韓国「ソウル新聞」は、翌11日（日本時...,0.0,1.0,0.0,0.0,1.0,0.000175,0.999705,0.000631,0.000178,0.999698
1,0.007343,0.985629,0.007028,data/text/topic-news/topic-news-6841012.txt,五輪サッカー日韓戦において、韓国が注目する“ある選手”とは？ 日本時間11日午前3時45分よ...,0.0,1.0,0.0,0.0,1.0,0.000175,0.999705,0.000631,0.000178,0.999698
2,0.007180,0.985483,0.007337,data/text/topic-news/topic-news-6829539.txt,女子アーチェリー銅メダリスト早川蓮、 韓国でも話題に 6日、アーチェリー女子銅メダリストの早...,0.0,1.0,0.0,0.0,1.0,0.000175,0.999705,0.001809,0.000188,0.999698
3,0.007506,0.984469,0.008026,data/text/topic-news/topic-news-6608684.txt,五輪逃したタイが「日本の八百長疑惑」を一斉報道 29日、タイでは、女子バレーボール五輪予選で...,0.0,1.0,0.0,0.0,1.0,0.000175,0.999705,0.001809,0.000188,0.999289
4,0.007822,0.984203,0.007975,data/text/topic-news/topic-news-6842234.txt,五輪サッカーで日韓が激突、韓国ユーザは「餌を出して試合させるのは間違っている」 韓国「文化日...,0.0,1.0,1.0,0.0,1.0,0.000175,0.999705,0.988512,0.000178,0.999698
5,0.007915,0.983509,0.008577,data/text/topic-news/topic-news-6818120.txt,女子バドミントン失格に韓国では「なでしこジャパンの引き分け狙いはOKなのになぜ…」の声 ロン...,0.0,1.0,0.0,0.0,1.0,0.000175,0.999705,0.001809,0.000178,0.999698
6,0.008171,0.983338,0.008491,data/text/topic-news/topic-news-6636116.txt,FIFA最新ランキング発表、韓国ユーザが敗北宣言!? 6日、FIFAはサッカーの世界ランキン...,0.0,1.0,0.0,0.0,1.0,0.000175,0.999705,0.000631,0.000178,0.999168
7,0.008762,0.982255,0.008983,data/text/topic-news/topic-news-6782959.txt,【韓国ニュース】WBC日本不参加の報道に「韓国の優勝か!?」 20日、労働組合・日本プロ野球...,0.0,1.0,0.0,0.0,1.0,0.000175,0.999705,0.001809,0.000235,0.999168
8,0.008930,0.981640,0.009430,data/text/topic-news/topic-news-6829622.txt,【五輪サッカー】 韓国に負けたイギリス代表に英ネットユーザーから批判相次ぐ 5日、イギリスの...,0.0,1.0,0.0,0.0,1.0,0.000175,0.999705,0.002167,0.000178,0.999698
9,0.009588,0.980599,0.009813,data/text/topic-news/topic-news-6822413.txt,五輪サッカー 英韓戦を前に韓国では「最悪のシナリオ」の声 2日、韓国のニュースサイト「イデ...,0.0,1.0,0.0,0.0,1.0,0.000175,0.999705,0.000631,0.000188,0.999289


In [15]:
# 予測は、　tag づけしていないデータにもつきます。
df_tag[100:]

,topic00,topic01,topic02,doc_file,doc_text,tag_芸能,tag_海外,tag_社会,tag_政治,tag_スポーツ,predict_tag_芸能,predict_tag_海外,predict_tag_社会,predict_tag_政治,predict_tag_スポーツ
100,0.017635,0.821989,0.160377,data/text/topic-news/topic-news-6017374.txt,日本の習慣に外国人が激怒 居酒屋で注文せずとも出される“お通し”。日本ではお馴染みの習慣だが...,NaN,NaN,NaN,NaN,NaN,0.000175,0.999705,0.005327,0.000178,0.000180
101,0.016988,0.821360,0.161651,data/text/topic-news/topic-news-5966020.txt,日本スケート連盟のキム・ヨナ締め出しに「当然」の声 「キム・ヨナが日本のアイスショーからは...,NaN,NaN,NaN,NaN,NaN,0.000175,0.999705,0.001653,0.000178,0.999168
102,0.010429,0.818316,0.171255,data/text/topic-news/topic-news-6388021.txt,中国警察が女子高生を生きたまま田んぼに遺棄 21日、朝日新聞は、中国の警察が倒れている女子高...,NaN,NaN,NaN,NaN,NaN,0.000175,0.999705,0.264500,0.999207,0.000180
103,0.014804,0.816421,0.168774,data/text/topic-news/topic-news-6226414.txt,「日本人が最も食べている漬物はキムチ」に「そんなわけないだろ」 25日、YOMIURI ON...,NaN,NaN,NaN,NaN,NaN,0.000175,0.999705,0.000631,0.000178,0.000180
104,0.008732,0.815220,0.176047,data/text/topic-news/topic-news-6395897.txt,中国提案の「南京大虐殺否定罪」に、日本では「幼稚で横暴」だと非難殺到 21日、NEWSポスト...,NaN,NaN,NaN,NaN,NaN,0.000175,0.999705,0.010239,0.999187,0.000180
105,0.010536,0.814796,0.174668,data/text/topic-news/topic-news-6705586.txt,【韓国ニュース】北朝鮮が日韓の軍事協定を批判 韓国ユーザーからは「敵の敵は味方」の声 28...,NaN,NaN,NaN,NaN,NaN,0.000175,0.999705,0.001809,0.000300,0.000180
106,0.014255,0.808532,0.177213,data/text/topic-news/topic-news-6349177.txt,バットの上に長時間正座…プロ野球界の主従関係を非難 7日、プロ野球独立リーグのBCリーグ「福...,NaN,NaN,NaN,NaN,NaN,0.000175,0.999705,0.001051,0.010447,0.999168
107,0.031740,0.807555,0.160705,data/text/topic-news/topic-news-6696339.txt,フィリピンで日本人女性2人に性的暴行を加えた韓国人を逮捕 韓国ネットユーザーからは「日本は昔...,NaN,NaN,NaN,NaN,NaN,0.000175,0.999705,0.300861,0.000235,0.000180
108,0.008011,0.803337,0.188652,data/text/topic-news/topic-news-6353761.txt,「勝手な広告出すな!」韓国への感謝広告に批判殺到 \t 9日、在韓国日本大使館が各韓国紙に「...,NaN,NaN,NaN,NaN,NaN,0.000175,0.999705,0.035103,0.999207,0.000180
109,0.004974,0.802660,0.192365,data/text/topic-news/topic-news-5934315.txt,ピザの起源は“韓国”とするCMが話題に ピザの起源を韓国だとする広告動画を韓国のピザ会社「ミ...,NaN,NaN,NaN,NaN,NaN,0.000175,0.999705,0.003605,0.000194,0.000180


In [16]:
# ファイルとして出力します。
file_name = 'data/tag_predict.csv'
df_tag.to_csv(file_name, index=False)


# 後は、、、

上記で出力された　data/tag_predict.csv　を目視で確認して、予測が間違っている部分の　tag を手動で入力し直しましょう。

（予測が正しい場合を、再入力してもよいですが、あまり効果はありません。現状の学習の強化になるだけです）

入力し直したデータを、　data/tag_predict.txt に上書きします。

そして、また再学習することで精度が強化されていきます。

こうして、「正しいデータを、効率的に増やしていきます」

後は、学習モデルをより高度なものに変更してみてもよいでしょう。

RNNでもBERTなど、自然言語の判別アルゴリズムは2019年の現在では頻繁にアップデートされています。

それらは、tagづけされたデータが出来上がってから改めて再学習することができます。